In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
import pandas as pd
import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from gensim.models import word2vec

Using TensorFlow backend.


In [2]:
# df1 = pd.read_csv('model_data.csv')

In [3]:
df = pd.read_csv('CLEAN_EDIT_model_data.csv')

In [4]:
df.drop(columns = ['Unnamed: 0'], inplace= True)

In [5]:
df.head()

,signal_x,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,ability,able,aboard,abort,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,stay,137,105,1,0.366667,assuming acceleration of to but in a comfortab...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,stay,64,113,1,0.200000,is capable of transporting satellite to orbit ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,stay,137,6,0,0.000000,yup,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,stay,137,7,0,0.000000,part,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,stay,137,96,1,0.650000,fly to most place on earth in under min and an...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df.tweet = df.tweet.astype(str)

In [7]:
target = df.signal_x

In [8]:
target

0       stay
1       stay
2       stay
3       stay
4       stay
5       stay
6       stay
7       stay
8       stay
9       stay
10      stay
11      stay
12      stay
13      stay
14      down
15      down
16      down
17      down
18      down
19      down
20      down
21      down
22      down
23      down
24      stay
25      down
26        up
27        up
28        up
29        up
        ... 
2172    stay
2173    stay
2174    stay
2175    stay
2176    stay
2177    stay
2178    stay
2179      up
2180      up
2181    stay
2182    stay
2183    stay
2184    stay
2185    stay
2186      up
2187      up
2188    stay
2189    stay
2190    stay
2191    down
2192    down
2193    stay
2194    stay
2195    stay
2196    stay
2197    stay
2198    stay
2199    stay
2200    stay
2201      up
Name: signal_x, Length: 2202, dtype: object

In [9]:
data = df['tweet'].map(word_tokenize).values

In [10]:
data

array([list(['assuming', 'acceleration', 'of', 'to', 'but', 'in', 'a', 'comfortable', 'direction', 'will', 'feel', 'like', 'a', 'mild', 'to']),
       list(['is', 'capable', 'of', 'transporting', 'satellite', 'to', 'orbit', 'crew', 'and', 'cargo', 'to', 'the', 'and', 'mission', 'to', 'the', 'moon', 'an']),
       list(['yup']), ...,
       list(['these', 'article', 'in', 'space', 'news', 'describe', 'why', 'v', 't', 'and', 't', 'w']),
       list(['wa', 'by', 'a', 'saying', 'rocket', 'ha', 'no', 'chance', 'just', 'said', 'the', 'franco', 'german', 'ha', 'no', 'chance', 'so', 'go', 'with']),
       list(['just', 'returned', 'from', 'a', 'trip', 'to', 'and', 'where', 'i', 'met', 'with', 'many', 'interesting', 'people', 'i', 'really', 'like'])],
      dtype=object)

In [11]:
type(data)

numpy.ndarray

In [12]:
y = pd.get_dummies(target).values

In [13]:
tokenizer = text.Tokenizer(num_words=15000)
tokenizer.fit_on_texts(list(df.tweet))
list_tokenized_tweets = tokenizer.texts_to_sequences(df.tweet)
X_t = sequence.pad_sequences(list_tokenized_tweets) #, maxlen=2709)

In [14]:
X_t.shape

(2202, 28)

In [15]:
embedding_size = 28
input_ = Input(shape=(28,))
x = Embedding(10000, embedding_size)(input_)
x = LSTM(25, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.5)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.5)(x)
# There are 41 different possible classes, so we use 41 neurons in our output layer
x = Dense(3, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)

In [16]:
# embedding_size = 128
# input_ = Input(shape=(100,))
# x = Embedding(10000, embedding_size)(input_)
# x = LSTM(25, return_sequences=True)(x)
# x = GlobalMaxPool1D()(x)
# x = Dropout(0.5)(x)
# x = Dense(50, activation='relu')(x)
# x = Dropout(0.5)(x)
# # There are 41 different possible classes, so we use 41 neurons in our output layer
# x = Dense(3, activation='relu')(x)

# model = Model(inputs=input_, outputs=x)

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 28)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 28, 28)            280000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 28, 25)            5400      
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 25)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 50)                1300      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
__________

In [ ]:
model.fit(X_t, y, epochs=20, batch_size=32, validation_split=0.1)

Train on 1981 samples, validate on 221 samples
Epoch 1/20


---

### New model experiments

#### GloVe

In [ ]:
total_vocabulary = set(word for tweet in data for word in tweet)
len(total_vocabulary)

In [ ]:
glove = {}
with open('glove.6B/glove.6B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [ ]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note from Mike: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # It can't be used in a sklearn Pipeline. 
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

In [ ]:
rf =  Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ("Random Forest", RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])

In [ ]:
models = [('Random Forest', rf),
          ("Support Vector Machine", svc),
          ("Logistic Regression", lr)]

In [ ]:
scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model, in models]

In [ ]:
scores

#### ANN Model


In [17]:
# ANN Model

In [15]:
df.head()

,signal_x,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,ability,able,aboard,abort,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,stay,137,105,1,0.366667,assuming acceleration of to but in a comfortab...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,stay,64,113,1,0.200000,is capable of transporting satellite to orbit ...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,stay,137,6,0,0.000000,yup,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,stay,137,7,0,0.000000,part,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,stay,137,96,1,0.650000,fly to most place on earth in under min and an...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_t, y, test_size = 0.2, random_state = 0)

In [17]:
val = X_train[:1300]
train_final = X_train[1300:]
label_val = y_train[:1300]
label_train_final = y_train[1300:]

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [19]:
classifier = Sequential()

In [20]:
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))

In [21]:
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))

In [22]:
classifier.add(Dense(units = 3, kernel_initializer='uniform', activation = 'softmax'))

In [23]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [24]:
classifier.fit(train_final, label_train_final, batch_size = 10, epochs = 100, validation_data=(val, label_val))

Train on 461 samples, validate on 1300 samples
Epoch 1/100
461/461 [==============================] - 2s 4ms/step - loss: 0.6535 - acc: 0.6551 - val_loss: 0.6407 - val_acc: 0.6651
Epoch 2/100
461/461 [==============================] - 0s 631us/step - loss: 0.6226 - acc: 0.6739 - val_loss: 0.6504 - val_acc: 0.6546
Epoch 3/100
461/461 [==============================] - 0s 751us/step - loss: 0.6107 - acc: 0.6797 - val_loss: 0.6531 - val_acc: 0.6485
Epoch 4/100
461/461 [==============================] - 0s 978us/step - loss: 0.5984 - acc: 0.6898 - val_loss: 0.6714 - val_acc: 0.6441
Epoch 5/100
461/461 [==============================] - 0s 679us/step - loss: 0.5910 - acc: 0.6941 - val_loss: 0.6599 - val_acc: 0.6495
Epoch 6/100
461/461 [==============================] - 0s 613us/step - loss: 0.5730 - acc: 0.7007 - val_loss: 0.7120 - val_acc: 0.6236
Epoch 7/100
461/461 [==============================] - 0s 698us/step - loss: 0.5692 - acc: 0.7014 - val_loss: 0.6988 - val_acc: 0.6326
Epoch 8/10

Epoch 61/100
461/461 [==============================] - 0s 620us/step - loss: 0.3342 - acc: 0.8445 - val_loss: 1.9445 - val_acc: 0.5831
Epoch 62/100
461/461 [==============================] - 0s 757us/step - loss: 0.2977 - acc: 0.8576 - val_loss: 1.8534 - val_acc: 0.5897
Epoch 63/100
461/461 [==============================] - 0s 669us/step - loss: 0.2862 - acc: 0.8662 - val_loss: 1.8667 - val_acc: 0.5864
Epoch 64/100
461/461 [==============================] - 0s 785us/step - loss: 0.2664 - acc: 0.8764 - val_loss: 1.9227 - val_acc: 0.5846
Epoch 65/100
461/461 [==============================] - 0s 666us/step - loss: 0.2532 - acc: 0.8814 - val_loss: 1.9475 - val_acc: 0.5882
Epoch 66/100
461/461 [==============================] - 0s 838us/step - loss: 0.2515 - acc: 0.8807 - val_loss: 1.9764 - val_acc: 0.5836
Epoch 67/100
461/461 [==============================] - 0s 620us/step - loss: 0.2721 - acc: 0.8604 - val_loss: 1.9575 - val_acc: 0.5808
Epoch 68/100
461/461 [==========================

In [25]:
results_train = classifier.evaluate(train_final, label_train_final)

461/461 [==============================] - 0s 40us/step


In [26]:
results_test = classifier.evaluate(X_test, y_test)

441/441 [==============================] - 0s 36us/step


In [27]:
results_train

[0.216625674815602, 0.8915401223941894]

In [28]:
results_test

[2.2477267215581707, 0.5638699868098408]

In [29]:
# import matplotlib.pyplot as plt
# plt.clf()

# loss_values = model_val_dict['loss']
# val_loss_values = model_val_dict['val_loss']

# epochs = range(1, len(loss_values) + 1)
# plt.plot(epochs, loss_values, 'g', label='Training loss')
# plt.plot(epochs, val_loss_values, 'blue', label='Validation loss')

# plt.title('Training & validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [30]:
score,acc = classifier.evaluate(X_test, y_test, verbose = 2, batch_size = 32)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 2.25
acc: 0.56


In [31]:
df = df.drop(['signal_x', 'tweet'], axis=1)

In [32]:
df

,retweet_from,tweet_length,encoded_sentiment,polarity,ability,able,aboard,abort,aborted,about,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,137,105,1,0.366667,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,64,113,1,0.200000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,137,6,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0
3,137,7,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,137,96,1,0.650000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5,64,80,1,0.125000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6,137,57,-1,-0.166667,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.271183,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7,137,84,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
8,137,16,-1,-0.800000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9,137,60,1,0.357143,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
X = df.as_matrix()

In [39]:
X

array([['stay', 137, 105, ..., 0.0, 0.0, 0.0],
       ['stay', 64, 113, ..., 0.0, 0.0, 0.0],
       ['stay', 137, 6, ..., 0.0, 0.0, 0.0],
       ...,
       ['stay', 137, 102, ..., 0.0, 0.0, 0.0],
       ['stay', 137, 136, ..., 0.0, 0.0, 0.0],
       ['up', 137, 112, ..., 0.0, 0.0, 0.0]], dtype=object)

In [43]:
type(target)

pandas.core.series.Series

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.2, random_state = 0)

In [41]:
classifier = Sequential()
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 3, kernel_initializer='uniform', activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, validation_split=0.1)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=      retweet_from  tweet_length  encoded_sentiment  polarity  ability  able  \
562             91           107                  1  0.350000      0.0   0.0   
1919           137           101                  1  0.150000      0.0   0.0   
1606           137           117                  0  0.000000      0.0   0.0   
1900           137           125                  1  0.200000      0.0   0.0   
1161            67            79                  1  0.240000      0.0   0.0   
279            137           117                  1  0.500000      0.0   0.0   
1519           137           137                  1  0.350000      0.0   0.0   
971            137            52                  0  0.000000      0.0   0.0   
1026           137            54                  0  0.000000      0.0   0.0   
310            137           104                  1  0.066667      0.0   0.0   
1021           137  